# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1: 
#   Name: Ronel Yust
#   I.D: 318434644
#   Mail: Ronelyust@gmail.com
# Details Student 2:
#   NONE

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#!pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

/Users/ronelyust/Desktop/Projects/Intro to Machine Learning/Assigment 2 - Model


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
# YOUR CODE HERE

# Importing the make_scorer function from scikit-learn's metrics module in order to create a custom scorer.
# The scorer will be used for evaluating the average f1 score later. 
from sklearn.metrics import make_scorer
 

# Extracting features and labels from training data and test data.
X_train = df_train['story']
y_train = df_train['gender']
X_test = df_test['story']

# Converting the gender labels 'f' and 'm' to numerical labels.
y_train = y_train.replace(['f'], 0)
y_train = y_train.replace(['m'], 1)

# Normalizing the text data using my normalize_hebrew_text function while making sure each letter is recognized in hebrew.
# Unrecongnized words will be switched with a space.
def normalize_hebrew_text(text):
    text = re.sub(r'[^\u0590-\u05FF\s]', '', text)
    text = ' '.join(text.split())
    
    return text
# Defining a custom transformer class HebrewTextNormalizer that implements the fit and transform methods.
# This class can be used in my pipeline to normalize Hebrew text without relying on external libraries.
class HebrewTextNormalizer:
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [normalize_hebrew_text(text) for text in X]


# Tokenizing the text data using my hebrew_tokenizer function while making sure each word is a recognized word in hebrew.
def hebrew_tokenizer(text):
    use_tokens = []
    tokens = ht.tokenize(text)
    for grp, token, token_num, (start_index, end_index) in tokens:
        if grp == 'HEBREW':
            use_tokens.append(token)
    return use_tokens 

# Creating a custom scorer function for macro average F1 score
f1_macro_scorer = make_scorer(f1_score, average='macro')

# Define the models by defining a list of tuples, each containing a model name, a model instance, and a parameter grid for hyperparameter tuning.
# The parameter grids specify the hyperparameter values for each model. 
models = [
        ('naive_bayes', MultinomialNB(), {'classifier__alpha': [0.01, 0.1, 1.0]}),
        ('svm', LinearSVC(), {
        'classifier__C': [0.1, 1.0],
        'vectorizer__max_features': [5000, 10000]
        }),
        ('neural_network', MLPClassifier(), {
            'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
            'classifier__activation': ['relu', 'tanh'],
            'classifier__alpha': [0.0001, 0.001, 0.01],
            'classifier__learning_rate_init': [0.001, 0.01, 0.1]
        }),
        ('neighbors', KNeighborsClassifier(), {
            'classifier__n_neighbors': [3, 5, 7],
            'classifier__weights': ['uniform', 'distance'],
        }),
        ('tree', DecisionTreeClassifier(), {
            'classifier__max_depth': [10, 20],
            'classifier__min_samples_split': [5, 10],
            'vectorizer__max_features': [5000, 10000]
        })
    ]

best_models = []

# Looping through each model in the models list, creating a pipeline that includes a TF-IDF vectorizer and the classifier.
# Using GridSearchCV to search over the hyperparameter grid for the best combination of hyperparameters that maximizes the macro average F1 score.
# The best hyperparameters, best average F1 score, and best estimator are stored. 
for model_name, model, param_grid in models:
        best_avg_f1 = 0
        best_params = None
        best_estimator = None

        pipeline = Pipeline([
            ('text_normalizer', HebrewTextNormalizer()),
            ('vectorizer', TfidfVectorizer(tokenizer=hebrew_tokenizer)),    
            ('classifier', model)
        ])

        grid_search = GridSearchCV(
            pipeline,
            param_grid,
            cv=10,
            scoring=f1_macro_scorer,
            refit=True,  # Refit with the best model based on the scorer
            return_train_score=True
        )

        grid_search.fit(X_train, y_train)

        # Calculating the best average F1 score using the macro average scorer
        avg_f1 = grid_search.best_score_

        if avg_f1 > best_avg_f1:
            best_avg_f1 = avg_f1
            best_params = grid_search.best_params_
            best_estimator = grid_search.best_estimator_

        best_models.append((model_name, best_avg_f1, best_params, best_estimator))

# Printing out the best models found during the grid search, along with their best average F1 scores and best parameters.
for model_name, best_avg_f1, best_params, best_estimator in best_models:
        print(f'Model: {model_name}')
        print(f'Best Average F1 score: {best_avg_f1:.3f}')
        print(f'Best Parameters: {best_params}')


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('text_normalizer',
                                        <__main__.HebrewTextNormalizer object at 0x7f92492ac220>),
                                       ('vectorizer',
                                        TfidfVectorizer(tokenizer=<function hebrew_tokenizer at 0x7f924929ca60>)),
                                       ('classifier', MultinomialNB())]),
             param_grid={'classifier__alpha': [0.01, 0.1, 1.0]},
             return_train_score=True,
             scoring=make_scorer(f1_score, average=macro))

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('text_normalizer',
                                        <__main__.HebrewTextNormalizer object at 0x7f92492afd30>),
                                       ('vectorizer',
                                        TfidfVectorizer(tokenizer=<function hebrew_tokenizer at 0x7f924929ca60>)),
                                       ('classifier', LinearSVC())]),
             param_grid={'classifier__C': [0.1, 1.0],
                         'vectorizer__max_features': [5000, 10000]},
             return_train_score=True,
             scoring=make_scorer(f1_score, average=macro))

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('text_normalizer',
                                        <__main__.HebrewTextNormalizer object at 0x7f92492ae170>),
                                       ('vectorizer',
                                        TfidfVectorizer(tokenizer=<function hebrew_tokenizer at 0x7f924929ca60>)),
                                       ('classifier', MLPClassifier())]),
             param_grid={'classifier__activation': ['relu', 'tanh'],
                         'classifier__alpha': [0.0001, 0.001, 0.01],
                         'classifier__hidden_layer_sizes': [(50,), (100,),
                                                            (50, 50),
                                                            (100, 50)],
                         'classifier__learning_rate_init': [0.001, 0.01, 0.1]},
             return_train_score=True,
             scoring=make_scorer(f1_score, average=macro))

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('text_normalizer',
                                        <__main__.HebrewTextNormalizer object at 0x7f926af839d0>),
                                       ('vectorizer',
                                        TfidfVectorizer(tokenizer=<function hebrew_tokenizer at 0x7f924929ca60>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid={'classifier__n_neighbors': [3, 5, 7],
                         'classifier__weights': ['uniform', 'distance']},
             return_train_score=True,
             scoring=make_scorer(f1_score, average=macro))

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('text_normalizer',
                                        <__main__.HebrewTextNormalizer object at 0x7f926903a320>),
                                       ('vectorizer',
                                        TfidfVectorizer(tokenizer=<function hebrew_tokenizer at 0x7f924929ca60>)),
                                       ('classifier',
                                        DecisionTreeClassifier())]),
             param_grid={'classifier__max_depth': [10, 20],
                         'classifier__min_samples_split': [5, 10],
                         'vectorizer__max_features': [5000, 10000]},
             return_train_score=True,
             scoring=make_scorer(f1_score, average=macro))

Model: naive_bayes
Best Average F1 score: 0.509
Best Parameters: {'classifier__alpha': 0.01}
Model: svm
Best Average F1 score: 0.599
Best Parameters: {'classifier__C': 1.0, 'vectorizer__max_features': 5000}
Model: neural_network
Best Average F1 score: 0.706
Best Parameters: {'classifier__activation': 'relu', 'classifier__alpha': 0.01, 'classifier__hidden_layer_sizes': (50,), 'classifier__learning_rate_init': 0.1}
Model: neighbors
Best Average F1 score: 0.602
Best Parameters: {'classifier__n_neighbors': 5, 'classifier__weights': 'distance'}
Model: tree
Best Average F1 score: 0.605
Best Parameters: {'classifier__max_depth': 10, 'classifier__min_samples_split': 5, 'vectorizer__max_features': 5000}


In [12]:
# YOUR CODE HERE

# Finding the model with the highest average F1 score from the list of best models and print its name, F1 score, and parameters.
best_model_info = max(best_models, key=lambda x: x[1])
best_model_name = best_model_info[0]
best_model_f1_score = best_model_info[1]
best_model_params = best_model_info[2]
best_model = best_model_info[3]

# Print the best model name and parameters
print(f'Best Model: {best_model_name}')
print(f'Best Model F1 Score: {best_model_f1_score}')
print(f'Best Model Parameters: {best_model_params}')

# Training the best model found on the full training data and making predictions on the test data using the trained model.
best_model.fit(X_train, y_train)
test_predictions = best_model.predict(X_test)

# Converting predictions back to 'm' and 'f'
test_predictions = ['f' if prediction == 0 else 'm' for prediction in test_predictions]

# Displaying predictions for the first and last 5 test examples
print("Predictions for the first 5 test examples:")
for i in range(5):
    print(f'Test Example {i+1}: Prediction - {test_predictions[i]}')

print("\nPredictions for the last 5 test examples:")
for i in range(-5, 0):
    print(f'Test Example {len(test_predictions) + i + 1}: Prediction - {test_predictions[i]}')

# Creating DataFrame for Predicted Results:
df_predicted = pd.DataFrame({
    'test_example_id': df_test['test_example_id'],
    'predicted_category': test_predictions
})

Best Model: neural_network
Best Model F1 Score: 0.7058763895621263
Best Model Parameters: {'classifier__activation': 'relu', 'classifier__alpha': 0.01, 'classifier__hidden_layer_sizes': (50,), 'classifier__learning_rate_init': 0.1}


Pipeline(steps=[('text_normalizer',
                 <__main__.HebrewTextNormalizer object at 0x7f926b7fb940>),
                ('vectorizer',
                 TfidfVectorizer(tokenizer=<function hebrew_tokenizer at 0x7f924929ca60>)),
                ('classifier',
                 MLPClassifier(alpha=0.01, hidden_layer_sizes=(50,),
                               learning_rate_init=0.1))])

Predictions for the first 5 test examples:
Test Example 1: Prediction - m
Test Example 2: Prediction - m
Test Example 3: Prediction - m
Test Example 4: Prediction - m
Test Example 5: Prediction - m

Predictions for the last 5 test examples:
Test Example 319: Prediction - m
Test Example 320: Prediction - m
Test Example 321: Prediction - m
Test Example 322: Prediction - m
Test Example 323: Prediction - m


In [13]:
# YOUR CODE HERE


In [14]:
# Add as many code cells as you need
# YOUR CODE HERE


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [15]:
df_predicted.to_csv('classification_results.csv',index=False)